# Module 5: Cost Optimization & Operational Excellence

## 🎯 Interactive Lab: Optimizing Redis Costs

**Duration:** 45 minutes  
**Level:** Intermediate  

In this lab, you'll:
- 💰 Understand Azure Redis pricing models
- 📊 Monitor memory usage and efficiency
- 🔍 Identify cost optimization opportunities
- ⚡ Implement memory-efficient patterns
- 📈 Calculate cost savings

---


## 🐳 Start Docker Redis Container

Before we begin, let's start a Redis container using Docker:

In [ ]:
# Start Redis container
!docker run -d \
  --name workshop-redis-module5 \
  -p 6379:6379 \
  redis:7-alpine

# Wait for Redis to be ready
import time
time.sleep(2)

# Test connection
!docker exec workshop-redis-module5 redis-cli ping

print('✅ Redis container is running on localhost:6379')

## Part 1: Setup


In [ ]:
!pip install -q redis

import redis
import sys
import json
import time

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Test connection
try:
    r.ping()
    print('✅ Connected to Redis')
except Exception as e:
    print(f'❌ Connection failed: {e}')
    print('   Make sure Redis is running')


---

## Part 2: Azure Redis Pricing Overview

### Pricing Tiers (Monthly Estimates)

| Tier | Size | Memory | Est. Cost/Month |
|------|------|--------|----------------|
| **Basic C1** | 1 GB | 1 GB | ~$18 |
| **Basic C2** | 2.5 GB | 2.5 GB | ~$37 |
| **Standard C1** | 1 GB | 1 GB | ~$55 |
| **Standard C2** | 2.5 GB | 2.5 GB | ~$110 |
| **Premium P1** | 6 GB | 6 GB | ~$237 |
| **Premium P2** | 13 GB | 13 GB | ~$503 |

### Cost Factors

1. **Tier Selection** - Basic vs Standard vs Premium
2. **Size** - Memory capacity
3. **Replication** - Additional replicas
4. **Persistence** - RDB/AOF snapshots
5. **Data Transfer** - Egress costs

### 💡 Cost Optimization Strategies

1. **Right-size your instance**
2. **Use appropriate data structures**
3. **Set TTLs on keys**
4. **Monitor memory usage**
5. **Use compression when beneficial**


---

## Part 3: Memory Usage Analysis

Let's analyze memory usage patterns:


In [ ]:
def get_memory_stats():
    """Get current memory statistics"""
    info = r.info('memory')
    
    used_mb = info['used_memory'] / 1024 / 1024
    peak_mb = info['used_memory_peak'] / 1024 / 1024
    rss_mb = info['used_memory_rss'] / 1024 / 1024
    
    return {
        'used_memory_mb': round(used_mb, 2),
        'peak_memory_mb': round(peak_mb, 2),
        'rss_memory_mb': round(rss_mb, 2),
        'fragmentation_ratio': info.get('mem_fragmentation_ratio', 1.0)
    }

# Get initial stats
stats = get_memory_stats()
print('📊 Current Memory Usage:')
print(f'   Used: {stats["used_memory_mb"]} MB')
print(f'   Peak: {stats["peak_memory_mb"]} MB')
print(f'   RSS: {stats["rss_memory_mb"]} MB')
print(f'   Fragmentation: {stats["fragmentation_ratio"]:.2f}')


---

## Part 4: Data Structure Efficiency

Different data structures have different memory footprints:


In [ ]:
import sys

def compare_data_structure_memory():
    """Compare memory usage of different approaches"""
    
    # Clear existing data
    r.flushdb()
    
    results = []
    
    # Approach 1: Individual string keys
    initial_mem = get_memory_stats()['used_memory_mb']
    
    for i in range(1000):
        r.set(f'user:{i}:name', f'User{i}')
        r.set(f'user:{i}:email', f'user{i}@example.com')
        r.set(f'user:{i}:age', 25 + (i % 50))
    
    string_mem = get_memory_stats()['used_memory_mb'] - initial_mem
    results.append(('Individual Strings (3000 keys)', string_mem))
    
    # Clean up
    r.flushdb()
    
    # Approach 2: Hashes
    initial_mem = get_memory_stats()['used_memory_mb']
    
    for i in range(1000):
        r.hset(f'user:{i}', mapping={
            'name': f'User{i}',
            'email': f'user{i}@example.com',
            'age': 25 + (i % 50)
        })
    
    hash_mem = get_memory_stats()['used_memory_mb'] - initial_mem
    results.append(('Hashes (1000 keys)', hash_mem))
    
    # Display results
    print('🔍 Memory Usage Comparison (1000 users, 3 fields each):')
    print()
    print(f'{"Approach":<30} | {"Memory (MB)":<12} | {"Savings"}')
    print('-' * 60)
    
    baseline = results[0][1]
    for approach, mem in results:
        savings = ((baseline - mem) / baseline * 100) if mem < baseline else 0
        print(f'{approach:<30} | {mem:>11.2f} | {savings:>5.1f}%')
    
    return results

# Run comparison
comparison = compare_data_structure_memory()


---

## Part 5: TTL Management for Cost Control

Setting Time-To-Live (TTL) prevents memory bloat:


In [ ]:
def demonstrate_ttl_impact():
    """Show impact of TTL on memory management"""
    
    # Clear database
    r.flushdb()
    
    print('📊 Creating cache entries...')
    
    # Scenario 1: No TTL (memory grows forever)
    print('\n❌ Without TTL:')
    for i in range(100):
        r.set(f'cache:no_ttl:{i}', f'data_{i}' * 100)
    
    no_ttl_keys = len(r.keys('cache:no_ttl:*'))
    print(f'   Keys: {no_ttl_keys}')
    print(f'   Will stay in memory indefinitely')
    
    # Scenario 2: With TTL (automatic cleanup)
    print('\n✅ With 60s TTL:')
    for i in range(100):
        r.setex(f'cache:with_ttl:{i}', 60, f'data_{i}' * 100)
    
    with_ttl_keys = len(r.keys('cache:with_ttl:*'))
    print(f'   Keys: {with_ttl_keys}')
    print(f'   Will auto-expire in 60 seconds')
    
    # Show memory stats
    stats = get_memory_stats()
    print(f'\n💾 Current memory usage: {stats["used_memory_mb"]} MB')
    
    # Calculate monthly cost impact
    print('\n�� Cost Impact (Example):')
    print('   Without TTL: Memory keeps growing → Requires larger instance')
    print('   With TTL: Memory auto-managed → Can use smaller instance')
    print('   Potential savings: $50-200/month per GB saved')

# Run demonstration
demonstrate_ttl_impact()


---

## Part 6: Right-Sizing Calculator

Determine the optimal Redis instance size:


In [ ]:
def estimate_redis_size(num_keys, avg_value_size_kb, overhead_factor=1.5):
    """
    Estimate required Redis memory
    
    Args:
        num_keys: Number of keys
        avg_value_size_kb: Average value size in KB
        overhead_factor: Multiplier for Redis overhead (default 1.5 = 50% overhead)
    """
    
    # Calculate raw data size
    raw_size_mb = (num_keys * avg_value_size_kb) / 1024
    
    # Add Redis overhead
    estimated_mb = raw_size_mb * overhead_factor
    
    # Add 20% buffer for growth
    recommended_mb = estimated_mb * 1.2
    
    # Suggest tier
    tiers = [
        ('C0 (250 MB)', 250, 15),
        ('C1 (1 GB)', 1024, 55),
        ('C2 (2.5 GB)', 2560, 110),
        ('C3 (6 GB)', 6144, 239),
        ('P1 (6 GB)', 6144, 237),
        ('P2 (13 GB)', 13312, 503),
        ('P3 (26 GB)', 26624, 1058),
    ]
    
    suggested_tier = None
    for tier_name, tier_mb, cost in tiers:
        if tier_mb >= recommended_mb:
            suggested_tier = (tier_name, cost)
            break
    
    print('🧮 Redis Size Calculator')
    print()
    print(f'📊 Your Workload:')
    print(f'   Keys: {num_keys:,}')
    print(f'   Avg value size: {avg_value_size_kb} KB')
    print()
    print(f'💾 Memory Estimates:')
    print(f'   Raw data: {raw_size_mb:.1f} MB')
    print(f'   With overhead: {estimated_mb:.1f} MB')
    print(f'   Recommended: {recommended_mb:.1f} MB (includes 20% growth buffer)')
    print()
    
    if suggested_tier:
        print(f'✅ Suggested Tier: {suggested_tier[0]}')
        print(f'   Estimated cost: ${suggested_tier[1]}/month')
    else:
        print('❌ Workload exceeds largest tier - consider clustering')

# Example 1: Small cache
print('Example 1: Small API Cache')
estimate_redis_size(num_keys=10000, avg_value_size_kb=2)

print('\n' + '='*60 + '\n')

# Example 2: Session store
print('Example 2: Session Store')
estimate_redis_size(num_keys=100000, avg_value_size_kb=5)


---

## Part 7: Memory Optimization Best Practices

### ✅ Do's

1. **Use Hashes for objects** - More memory efficient than separate keys
2. **Set TTLs on temporary data** - Prevent unbounded growth
3. **Monitor memory usage** - Set up alerts at 80% capacity
4. **Use appropriate data structures** - Sorted Sets for rankings, Hashes for objects
5. **Compress large values** - For values > 100KB

### ❌ Don'ts

1. **Don't store very large values** - Keep values < 1MB
2. **Don't use Redis as primary database** - Use as cache/session store
3. **Don't skip maxmemory policy** - Set eviction policy
4. **Don't ignore fragmentation** - Monitor and address
5. **Don't over-provision** - Start small, scale up as needed


## Cleanup


In [ ]:
# Clean up test data
r.flushdb()
print('✅ Redis data cleaned')

# Stop and remove Docker container
!docker stop workshop-redis-module5
!docker rm workshop-redis-module5

print('✅ Docker container removed')
print('✅ Cleanup complete')

---

## 🎯 Key Takeaways

### 💰 Cost Optimization

1. **Right-Size Your Instance**
   - Start with smaller tier
   - Monitor memory usage
   - Scale up only when needed

2. **Memory Efficiency**
   - Use Hashes instead of separate keys (30-50% savings)
   - Set TTLs on cache entries
   - Choose appropriate data structures

3. **Monitoring**
   - Track memory usage trends
   - Set alerts at 80% capacity
   - Review eviction metrics

4. **Cost Calculation**
   - Every 1 GB saved = ~$50-80/month
   - Proper TTLs can reduce costs by 30-50%
   - Right data structures save 20-40% memory

### 🔧 Optimization Checklist

- ✅ Set maxmemory and eviction policy
- ✅ Use Hashes for multi-field objects
- ✅ Set TTLs on all cache keys
- ✅ Monitor memory usage
- ✅ Regular memory analysis
- ✅ Review and optimize key patterns

---

## 🎉 Great Job!

You now know how to optimize Redis costs and memory usage!
